# Tier 1: probing each target prefixes

In [1]:
import logging
import pickle
import json

from pathlib import Path
import uuid

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.query_api import get_measurements_from_tag

from geoloc_imc_2023.default import (
    PROBES_PATH,
    ANCHORS_PATH,
    HITLIST_PATH,
    RIPE_CREDENTIALS,
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 2
NB_VP = 5

## load datasets

In [2]:
# load hitlist
with open(HITLIST_PATH, "rb") as f:
    targets_per_prefix = pickle.load(f)

# load probes
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_PATH, "rb") as f:
    anchors = pickle.load(f)

## select target and vps dataset

In [3]:
# select targets and vps from anchors
targets= list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 3
INFO:root:nb_vps : 10


## measure target prefixes

In [4]:
dry_run = True
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

cbg = CBG(RIPE_CREDENTIALS)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# measurement for 3 targets in every target prefixes
measurement_config["start_time"], measurement_config["end_time"] = cbg.prefix_probing(
    target_prefixes= target_prefixes, 
    vps= vps,
    targets_per_prefix= targets_per_prefix,
    tag= measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

INFO:root:starting measurement for target_prefix='213.225.160.0' with ['213.225.160.159', '213.225.160.140', '213.225.160.14'] with 10 vps
INFO:root:starting measurement for target_prefix='145.220.0.0' with ['145.220.0.1', '145.220.0.28', '145.220.0.29'] with 10 vps
INFO:root:Reached limit for number of conccurent measurements: 5 (limit is 5)
INFO:root:starting measurement for target_prefix='5.28.0.0' with ['5.28.0.6', '5.28.0.13', '5.28.0.14'] with 10 vps
INFO:root:measurement : e6000fd2-d187-4947-99ed-8b1cf36f2d8c done


## Retrieve prefix probing results 

In [ ]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

# test that everything is alright
logging.info(f"nb measurements retrieved: {len(measurement_results)}")

print(f"measurements for {len(measurement_results)} target addr.")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10: break
    print(target_addr, ":", len(results))

# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)
    

# Tier 2: probe each target

In [4]:
import logging
import pickle
import json
import uuid

from pathlib import Path
from random import choice

from geoloc_imc_2023.query_api import get_measurements_from_tag, get_measurement_from_id
from geoloc_imc_2023.cbg import CBG
from geoloc_imc_2023.default import (
    PROBES_PATH,
    ANCHORS_PATH,
    RIPE_CREDENTIALS,
)

logging.basicConfig(level=logging.DEBUG)


NB_TARGET = 10
NB_VP = 30

## load data

In [5]:
# load probes
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_PATH, "rb") as f:
    anchors = pickle.load(f)

## get targets and vps datasets

In [9]:
# select targets and vps from anchors
targets= set()
while len(targets) < NB_TARGET:
    random_target = choice(list(anchors))
    targets.add(random_target)
targets = list(targets)

# select vps from probes
vps = {}
while len(vps) < NB_VP:
    random_vp = choice(list(probes))
    vps[random_vp] = probes[random_vp]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 10
INFO:root:nb_vps : 30


## measure targets

In [11]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f)

cbg = CBG(RIPE_CREDENTIALS)

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = cbg.target_probing(
    targets= targets, 
    vps= vps,
    tag= measurement_uuid,
    dry_run=dry_run
)

INFO:root:Reached limit for number of concurrent measurements: 5
INFO:root:Reached limit for number of concurrent measurements: 5
INFO:root:measurement : db5b34f9-edef-4c08-a0fc-332106015718 done


In [12]:
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

## retrieve measurement results

In [13]:
from geoloc_imc_2023.query_api import get_measurements_from_tag, get_measurement_url
import time
import requests

def get_from_atlas(url: str, max_retry: int = 60):
    """request to Atlas API"""

    for _ in range(max_retry):
        response = requests.get(url, timeout=20).json()
        print(response)
        if response:
            break
        time.sleep(2)

    return response

for measurement_id in measurement_config["ids"]: 
    logging.info(f"getting measurement with id : {measurement_id}")

measurement_id = 52712915
url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/"
print(url)
response = requests.get(url, timeout=20).json()
logging.info(f"RESP: {response['status']}")
logging.info(f"RESP: {response['result']}")

result = requests.get(response['result'])
print(result.content)


INFO:root:getting measurement with id : 52717385
INFO:root:getting measurement with id : 52717387
INFO:root:getting measurement with id : 52717389
INFO:root:getting measurement with id : 52717392
INFO:root:getting measurement with id : 52717393
INFO:root:getting measurement with id : 52717416
INFO:root:getting measurement with id : 52717418
INFO:root:getting measurement with id : 52717419
INFO:root:getting measurement with id : 52717420
INFO:root:getting measurement with id : 52717421


https://atlas.ripe.net/api/v2/measurements/52712915/


INFO:root:RESP: {'id': 4, 'name': 'Stopped', 'when': 1682590507}
INFO:root:RESP: https://atlas.ripe.net/api/v2/measurements/52712915/results/


b'[{"fw":5080,"mver":"2.6.2","lts":11,"dst_name":"217.70.176.109","af":4,"dst_addr":"217.70.176.109","src_addr":"10.71.71.15","proto":"ICMP","ttl":50,"size":48,"result":[{"rtt":167.635967},{"rtt":167.593986},{"rtt":167.989166}],"dup":0,"rcvd":3,"sent":3,"min":167.593986,"max":167.989166,"avg":167.7397063333,"msm_id":52712915,"prb_id":1000927,"timestamp":1682590165,"msm_name":"Ping","from":"103.38.202.251","type":"ping","group_id":52712915,"step":null,"stored_timestamp":1682598256},{"fw":5080,"mver":"2.6.2","lts":9,"dst_name":"217.70.176.109","af":4,"dst_addr":"217.70.176.109","src_addr":"192.168.178.49","proto":"ICMP","ttl":47,"size":48,"result":[{"rtt":67.862515},{"rtt":68.294253},{"rtt":65.89308}],"dup":0,"rcvd":3,"sent":3,"min":65.89308,"max":68.294253,"avg":67.3499493333,"msm_id":52712915,"prb_id":22312,"timestamp":1682590164,"msm_name":"Ping","from":"89.144.206.159","type":"ping","group_id":52712915,"step":null,"stored_timestamp":1682599993},{"fw":5080,"mver":"2.6.2","lts":6,"dst_

In [14]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

if not measurement_results:
    for measurement_id in measurement_config["ids"]: 
        logging.info(f"getting measurement with id : {measurement_id}")
        measurement_result = get_measurement_from_id(measurement_id)

logging.info(f"measurements for {len(measurement_results)} target addr")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10: break
    logging.info(f" {target_addr} : number of measurements {len(results)}")

# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
logging.info(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

INFO:root:measurements for 10 target addr
INFO:root: 188.94.96.182 : number of measurements 4
INFO:root: 104.167.214.46 : number of measurements 4
INFO:root: 85.236.41.237 : number of measurements 4
INFO:root: 185.97.141.2 : number of measurements 4
INFO:root: 92.38.132.60 : number of measurements 4
INFO:root: 95.217.19.69 : number of measurements 4
INFO:root: 204.9.92.179 : number of measurements 4
INFO:root: 54.39.185.31 : number of measurements 4
INFO:root: 80.79.16.242 : number of measurements 4
INFO:root: 195.32.69.197 : number of measurements 4
INFO:root:../results/db5b34f9-edef-4c08-a0fc-332106015718.pickle
